# Cleaning data

#### Hello!

This analysis aims to address the following business problem: A new e-commerce shop plans to sell laptops and accessories. They are interested in predicting laptop prices to optimize the price difference between wholesale purchase and selling price, as well as to identify the hardware parameters that influence the final price.

This notebook focuses on the original data file obtained from the Kaggle website. The data was scraped from various online stores by the author. The dataset contains numerous laptop items with different features and prices, but it requires cleaning and additional feature engineering before analysis.

The main objectives of this notebook are to tidy up the data, enhance it by deriving new features from the existing ones, and ultimately save the cleaned dataset to a new file. This new file will be ready for further analysis and building machine learning models.

In [26]:
# importing basic librarys

import numpy as np
import pandas as pd

In [27]:
# loading, reviewing data and checking types

data = pd.read_csv('.\\data\\laptopData.csv')
data.sample(10)

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
472,472.0,Google,Ultrabook,12.3,Touchscreen 2400x1600,Intel Core i5 7Y57 1.2GHz,8GB,128GB SSD,Intel HD Graphics 615,Chrome OS,1.1kg,67932.000
1098,1098.0,HP,Notebook,15.6,1366x768,Intel Pentium Quad Core N3710 1.6GHz,4GB,1TB HDD,Intel HD Graphics 405,Windows 10,1.96kg,26640.000
935,935.0,HP,Netbook,12.5,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.26kg,88924.320
344,344.0,Dell,Ultrabook,13.3,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD,Intel UHD Graphics 620,Windows 10,1.2kg,84129.120
85,85.0,Dell,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,Windows 10,2.65kg,71874.720
157,157.0,Asus,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.1kg,70063.200
1132,1132.0,Asus,Notebook,14,1366x768,Intel Celeron Dual Core N3350 1.1GHz,4GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.5kg,14811.840
462,462.0,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.37kg,39906.720
340,340.0,Dell,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics,Linux,2.2kg,14646.672
1152,1152.0,Lenovo,Notebook,15.6,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,1TB HDD,AMD Radeon R5 M330,No OS,2.32kg,31914.720


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1273 non-null   float64
 1   Company           1273 non-null   object 
 2   TypeName          1273 non-null   object 
 3   Inches            1273 non-null   object 
 4   ScreenResolution  1273 non-null   object 
 5   Cpu               1273 non-null   object 
 6   Ram               1273 non-null   object 
 7   Memory            1273 non-null   object 
 8   Gpu               1273 non-null   object 
 9   OpSys             1273 non-null   object 
 10  Weight            1273 non-null   object 
 11  Price             1273 non-null   float64
dtypes: float64(2), object(10)
memory usage: 122.3+ KB


In [29]:
# Delete column with index.

data = data.drop('Unnamed: 0', axis=1)

In [30]:
# Cleaning data:
# - removing the ends ('kg', 'GB', 'Ghz')
# - adding features: Resolution_type, Cpu_speed, Cpu_mark, Memory_GB, Meomory_kind
# - correct features Ram, Gpu, ScreenResolution
# - converting numeric features on numeric type
# - setting a NaN value for unknown values

import re

pattern_1 = r'\b\d+x\d+\b'


def return_resolution_type(sr: str) -> str:
    if pd.isnull(sr):
        return np.nan
    if 'Full HD' in sr:
        resolution_type = 'Full HD'
    elif 'Quad HD' in sr:
        resolution_type = 'Quad HD'    
    elif '4K Ultra HD' in sr:
        resolution_type = '4K Ultra HD'
    elif 'Retina Display' in sr:
        resolution_type = 'Retina Display'
    elif '1366x768 in sr':
        resolution_type = 'HD'
    else:
        resolution_type = 'other'
    return resolution_type


def return_resolution(sr: str) -> str:
    if pd.isnull(sr):
        return np.nan
    resolution = re.search(pattern_1, sr)
    return resolution.group()


def drop_GB(ram: str) -> float:
    if pd.isnull(ram) or ram == '':
        return np.nan
    return float(ram.rstrip('GB'))


def drop_kg(weight: str) -> float:
    if pd.isnull(weight) or weight == '?':
        return np.nan    
    return float(weight.rstrip('kg'))


def find_speed_cpu(cpu: str) -> float:
    if pd.isnull(cpu):
        return np.nan
    cpu_list = cpu.split()
    return float(cpu_list[-1].rstrip('GHz'))


def find_mark_cpu(cpu:str) -> str:
    if pd.isnull(cpu):
        return np.nan
    cpu_list = cpu.split()
    return cpu_list[0]


def find_size_memory(memory: str) -> float:
    ''' 
    The function returns the total memory as an integer in GB units.

    Example: '256GB SSD + 1TB HDD' -> 1280.0
    '''
    if pd.isnull(memory):
        return np.nan
    total_memory = 0
    memory_split = memory.split(sep='+')
    for memory_element in memory_split:
        memory_size = memory_element.split()
        if memory_size[0].endswith('TB'):
            memory_gb = 1024*float(memory_size[0].replace('TB', ''))
            total_memory += memory_gb
        elif memory_size[0].endswith('GB'):
            memory_gb = float(memory_size[0].replace('GB', ''))
            total_memory += memory_gb
        else:
            total_memory = np.nan
    return total_memory

    
def find_kind_memory(memory: str) -> str:
    if pd.isnull(memory):
        return np.nan
    if 'SSD' in memory:
        memory_kind = 'SSD'
    elif 'HDD' in memory:
        memory_kind = 'HDD'
    elif 'Flash' in memory:
        memory_kind = 'Flash'
    elif 'Hybrid' in memory:
        memory_kind = 'Hybrid'
    else:
        memory_kind = np.nan
    return memory_kind


def find_touchscreen(sr: str) -> str:
    if pd.isnull(sr):
        return np.nan
    if 'touchscreen' in sr.lower():
        touchscreen = 'Yes'
    else:
        touchscreen = 'No'
    return touchscreen


def find_ips(sr: str) -> str:
    if pd.isnull(sr):
        return np.nan
    if 'ips panel' in sr.lower():
        ips = 'Yes'
    else:
        ips = 'No'
    return ips

def find_mark_gpu(gpu: str) -> str:
    if pd.isnull(gpu):
        return np.nan
    if 'Intel' in gpu:
        gpu_mark = 'Intel'
    elif 'Nvidia' in gpu:
        gpu_mark = 'Nvidia'
    elif 'AMD' in gpu:
        gpu_mark = 'AMD'
    elif 'ARM' in gpu:
        gpu_mark = 'ARM'
    else:
        gpu_mark = np.nan
    return gpu_mark


data = data.assign(
    Ram=lambda x: x['Ram'].map(drop_GB),
    Weight=lambda x: x['Weight'].map(drop_kg),
    ResolutionType=lambda x: x['ScreenResolution'].map(return_resolution_type),
    Touchscreen=lambda x: x['ScreenResolution'].map(find_touchscreen),
    IPS=lambda x: x['ScreenResolution'].map(find_ips),
    ScreenResolution=lambda x: x['ScreenResolution'].map(return_resolution),
    CpuSpeed=lambda x: x['Cpu'].map(find_speed_cpu),
    CpuMark=lambda x: x['Cpu'].map(find_mark_cpu),
    MemoryGB=lambda x: x['Memory'].map(find_size_memory),
    MemoryKind=lambda x: x['Memory'].map(find_kind_memory),
    Gpu=lambda x: x['Gpu'].map(find_mark_gpu)
    )

data.head(3)

data = data[data['Inches'] != '?']
data['Inches'] = data['Inches'].astype(float)

data.drop(columns='Cpu',inplace=True)
data.drop(columns='Memory',inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 1302
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           1272 non-null   object 
 1   TypeName          1272 non-null   object 
 2   Inches            1272 non-null   float64
 3   ScreenResolution  1272 non-null   object 
 4   Ram               1272 non-null   float64
 5   Gpu               1272 non-null   object 
 6   OpSys             1272 non-null   object 
 7   Weight            1271 non-null   float64
 8   Price             1272 non-null   float64
 9   ResolutionType    1272 non-null   object 
 10  Touchscreen       1272 non-null   object 
 11  IPS               1272 non-null   object 
 12  CpuSpeed          1272 non-null   float64
 13  CpuMark           1272 non-null   object 
 14  MemoryGB          1271 non-null   float64
 15  MemoryKind        1271 non-null   object 
dtypes: float64(6), object(10)
memory usage: 172.9+ 

In [31]:
# checking how many records in each class contain the value Nan

data.isnull().sum()

Company             30
TypeName            30
Inches              30
ScreenResolution    30
Ram                 30
Gpu                 30
OpSys               30
Weight              31
Price               30
ResolutionType      30
Touchscreen         30
IPS                 30
CpuSpeed            30
CpuMark             30
MemoryGB            31
MemoryKind          31
dtype: int64

In [32]:
# let's randomly check 10 records to see if entire records are filled with NaN values or only individual cells.

data[data.isna().any(axis=1)].sample(10)

,Company,TypeName,Inches,ScreenResolution,Ram,Gpu,OpSys,Weight,Price,ResolutionType,Touchscreen,IPS,CpuSpeed,CpuMark,MemoryGB,MemoryKind
478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,Dell,Ultrabook,13.3,1920x1080,8.0,Intel,Windows 10,NaN,77202.72,Full HD,No,No,1.8,Intel,256.0,SSD


In [33]:
# We can see that these are mostly complete empty records. 
# Therefore, we will remove them. We will remove a total of 30 records, so we won't lose a significant amount of data.
# Removing all records where was 'NaN' and check summary.

data = data.dropna()
data.isnull().sum()

Company             0
TypeName            0
Inches              0
ScreenResolution    0
Ram                 0
Gpu                 0
OpSys               0
Weight              0
Price               0
ResolutionType      0
Touchscreen         0
IPS                 0
CpuSpeed            0
CpuMark             0
MemoryGB            0
MemoryKind          0
dtype: int64

In [34]:
# Reordering the columns in the DataFrame so that the 'Price' column is last.

column_order = list(data.columns)
column_order.remove("Price")
column_order.append("Price")
data = data.reindex(columns=column_order)
data.head(10)

,Company,TypeName,Inches,ScreenResolution,Ram,Gpu,OpSys,Weight,ResolutionType,Touchscreen,IPS,CpuSpeed,CpuMark,MemoryGB,MemoryKind,Price
0,Apple,Ultrabook,13.3,2560x1600,8.0,Intel,macOS,1.37,Retina Display,No,Yes,2.3,Intel,128.0,SSD,71378.6832
1,Apple,Ultrabook,13.3,1440x900,8.0,Intel,macOS,1.34,HD,No,No,1.8,Intel,128.0,Flash,47895.5232
2,HP,Notebook,15.6,1920x1080,8.0,Intel,No OS,1.86,Full HD,No,No,2.5,Intel,256.0,SSD,30636.0000
3,Apple,Ultrabook,15.4,2880x1800,16.0,AMD,macOS,1.83,Retina Display,No,Yes,2.7,Intel,512.0,SSD,135195.3360
4,Apple,Ultrabook,13.3,2560x1600,8.0,Intel,macOS,1.37,Retina Display,No,Yes,3.1,Intel,256.0,SSD,96095.8080
5,Acer,Notebook,15.6,1366x768,4.0,AMD,Windows 10,2.10,HD,No,No,3.0,AMD,500.0,HDD,21312.0000
6,Apple,Ultrabook,15.4,2880x1800,16.0,Intel,Mac OS X,2.04,Retina Display,No,Yes,2.2,Intel,256.0,Flash,114017.6016
7,Apple,Ultrabook,13.3,1440x900,8.0,Intel,macOS,1.34,HD,No,No,1.8,Intel,256.0,Flash,61735.5360
8,Asus,Ultrabook,14.0,1920x1080,16.0,Nvidia,Windows 10,1.30,Full HD,No,No,1.8,Intel,512.0,SSD,79653.6000
9,Acer,Ultrabook,14.0,1920x1080,8.0,Intel,Windows 10,1.60,Full HD,No,Yes,1.6,Intel,256.0,SSD,41025.6000


In [36]:
data.to_csv(".\\data\\cleaning_data.csv", index=False)

We are finishing the data cleaning process. Behind us is the addition of several new features based on the existing ones, changing the data type in numerical columns and deleting records with missing data. In the next notebook "price_laptops_analyst", we will deal with data visualization and building regression models.